In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import keras
from PIL import Image
from imgaug import augmenters as iaa
from keras.applications.xception import Xception
from keras.models import Model

In [14]:
BASE_DIR = os.path.dirname(os.path.abspath(''))
#look fol folder with images
image_dir = os.path.join(BASE_DIR, "awe")
#iterate trough images
PATH = "awe"
total_subjects = 100
epochs = 100 
            

In [3]:
def dataset(mode): 
    total_images = []
    label =[]
    mode_path = os.path.join(PATH,mode)  # mode path -> awe/TrainData
    subjects = os.listdir(mode_path)  # 
    
    for subject in subjects:
        image_path  = os.path.join(mode_path,subject)
        images = os.listdir(image_path)
        for image in images:
            if(image.endswith(".png")):
                file = os.path.join(image_path,image)
                total_images.append(cv2.resize(cv2.imread(file),(100,100)))
                label.append(int(subject))
    total_images = np.array(total_images)
    label = np.array(label)
    return total_images , label

In [4]:
train,label = dataset('TrainData')

In [5]:
def split_data(data,label,valid_len):
    valid_len = int(valid_len*len(data)/100)
    return (data[0:len(data)-valid_len],label[0:len(data)-valid_len],
            data[len(data)-valid_len:len(data)],label[len(data)-valid_len:len(data)])

In [6]:
x_train,y_train,x_valid,y_valid = split_data(train,label,20)

In [7]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(800, 100, 100, 3)
(200, 100, 100, 3)
(800,)
(200,)


In [8]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, total_subjects+1,dtype='int32')
y_valid = keras.utils.to_categorical(y_valid, total_subjects+1,dtype='int32')

In [9]:
y_train = y_train[:,0:total_subjects]
y_valid = y_valid[:,0:total_subjects]

print(y_train[0])
print(y_valid[0])

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [10]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(800, 100, 100, 3)
(200, 100, 100, 3)
(800, 100)
(200, 100)


In [11]:
#create model and fit the model
base_model = Xception(include_top=False, weights='imagenet',input_shape=x_train[0].shape)

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
predictions = keras.layers.Dense(total_subjects,activation='softmax')(x)

model = Model(base_model.input,predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 49, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 49, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 49, 32)   0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [12]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=70,
          epochs=epochs,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/100
12/12 [==============================] - 3s 230ms/step - loss: 4.6654 - accuracy: 0.0063 - val_loss: 4.5586 - val_accuracy: 0.0000e+00
Epoch 2/100
12/12 [==============================] - 3s 228ms/step - loss: 4.6506 - accuracy: 0.0125 - val_loss: 4.5000 - val_accuracy: 0.0050
Epoch 3/100
12/12 [==============================] - 3s 227ms/step - loss: 4.6377 - accuracy: 0.0125 - val_loss: 4.4844 - val_accuracy: 0.0000e+00
Epoch 4/100
12/12 [==============================] - 3s 225ms/step - loss: 4.6468 - accuracy: 0.0100 - val_loss: 4.4648 - val_accuracy: 0.0050
Epoch 5/100
12/12 [==============================] - 3s 228ms/step - loss: 4.6357 - accuracy: 0.0088 - val_loss: 4.4439 - val_accuracy: 0.0050
Epoch 6/100
12/12 [==============================] - 3s 229ms/step - loss: 4.6374 - accuracy: 0.0063 - val_loss: 4.4381 - val_accuracy: 0.0100
Epoch 7/100
12/12 [==============================] - 3s 229ms/step - loss: 4.6204 - accuracy: 0.0100 - val_loss: 4.4341 - val_accuracy

Epoch 58/100
12/12 [==============================] - 3s 233ms/step - loss: 4.3847 - accuracy: 0.0575 - val_loss: 4.4906 - val_accuracy: 0.0100
Epoch 59/100
12/12 [==============================] - 3s 235ms/step - loss: 4.3766 - accuracy: 0.0587 - val_loss: 4.4921 - val_accuracy: 0.0100
Epoch 60/100
12/12 [==============================] - 3s 233ms/step - loss: 4.3646 - accuracy: 0.0675 - val_loss: 4.4938 - val_accuracy: 0.0100
Epoch 61/100
12/12 [==============================] - 3s 232ms/step - loss: 4.3611 - accuracy: 0.0750 - val_loss: 4.4957 - val_accuracy: 0.0100
Epoch 62/100
12/12 [==============================] - 3s 232ms/step - loss: 4.3803 - accuracy: 0.0613 - val_loss: 4.4971 - val_accuracy: 0.0100
Epoch 63/100
12/12 [==============================] - 3s 233ms/step - loss: 4.3556 - accuracy: 0.0725 - val_loss: 4.4996 - val_accuracy: 0.0100
Epoch 64/100
12/12 [==============================] - 3s 236ms/step - loss: 4.3485 - accuracy: 0.0625 - val_loss: 4.5009 - val_accuracy:

In [ ]:
#results = np.argmax(model.predict(x_test),axis = 1)
#results = np.reshape(results,(len(results),1))
#print("Accuracy: ",(sum(results == y_test))/len(results) * 100)